# Ellipsoid Method of Feasibility

The point of this code is to find a feasible point of a polygon $P = \{x \in \mathbb{R}^n: Ax \geq b\}$ where $A$ is an $m \times n$ matrix and $b$ is a vector of length $m$ 

In [2]:
import numpy as np
import math
import matplotlib.pyplot as plt

from numpy import linalg

## Misc. Functions

In [3]:
def sign(num):
    if num < 0:
        return -1
    else:
        return 1

In [4]:
# Gives the vertical transpose of a horizontal vector. Used in finding new center and ellipse.
def trans(row):
    new= []
    for element in range(len(row)):
        new.append([row[element]])
        
    return np.array(new)

In [5]:
# Graphs a 2D ellipse of the form E(c,D)
def graphEllipse(D,c):
    x = np.linspace(-10,10, 1000)
    y = np.linspace(-10,10, 1000)
    x, y = np.meshgrid(x, y)
    
    inD = np.linalg.inv(D)
    
    plt.contour(x,y,((inD[0][0]*(x-c[0])**2) + ((inD[0][1]+inD[1][0])*(y-c[1])*(x-c[0])) + (inD[1][1]*(y-c[1])**2)), [1], colors='blue')
    plt.plot(c[0],c[1], marker='o', color='red')
    plt.show()

## Initialize

In [6]:
# This function find the correct length of our solution vector x
def findDimension(A):
    return(float(len(A[0])))

In [7]:
# This function finds the largest item in the given matrix A and vector b, so it can be used in the initialization
def findU(A,b):
    List_of_Elements = []
    
    for row in A:
        for element in row:
            List_of_Elements.append(element)
            
    for row in b:
        for element in row:
            List_of_Elements.append(element)
            
    return(max(List_of_Elements))

In [8]:
# Generates the initial values needed maximum runtime, and our first ellipse. Radius given for graphing purposes
def Initialize(A,b):
    n = findDimension(A)
    U = findU(A,b)
    
    # The smallest possible volume of the space
    v = ((n**(-n)) * ((n*U)**(-(n**2)*(n+1))))
    
    
    # The volume for which the must be a feasible point if one exists
    W = (((2*n)**n) * (n * U)**(n**2))
    
    # Maximum number of iterations needed
    Max_Time = math.ceil(2*(n+1) * math.log(W/v))
    
    # Gives the initial center as the origin
    Center = np.zeros((int(n),1))
    
    # the radius squared of the initial circle
    r2 = W/math.pi
    
    # Original Circle matrix
    D = r2*np.identity(int(n))
    
    return Max_Time, Center, D, r2

## Main Loop

In [9]:
# Give the expectation value for a given vector x and matrix A: x^T A x. Used in finding a new center and ellipse
def expectation(A,x):
    return x.dot(A.dot(trans(x)))[0]

In [10]:
# gives A x x^T A, a new matrix for a new ellipse
def newMatrix(A,x):
    return A.dot(trans(x))*x.dot(A)

In [42]:
def NewCenter(n,a,D,c):
    return c + (1/(n+1)) * ((D.dot(trans(a)))/(math.sqrt(expectation(D,a))))

In [47]:
def NewEllipseMatrix(n,a,D,c):
    return (D - ((2/(n+1)) * (newMatrix(D,a)/expectation(D,a))))

## Main Function

In [44]:
def FeasibilityEllipsoidMethod(A,b):
    n = findDimension(A)
    time = 0
    # Initialize
    Max_Time, center, D, r2 = Initialize(A,b)

    index = 0
    
    # Goes through each row of A, to find if one has been violated
    while index < len(A):
        if time >= Max_Time:
            return print('The solution space is empty.')
        
        if A[index].dot(center) >= b[index][0]:
                index = index + 1
        else:
            center = NewCenter(n,A[index],D,center)
            D = NewEllipseMatrix(n,A[index],D,center)
            
            time = time+1
            index = 0
            
            # Will Graph each step if desired
            #graphEllipse(D,center)
    
    return print(f'A valid solution is: \n {center}')

## Testing

In [45]:
A = np.array([[1,0,0],
               [0,1,0],
               [-1,-1,0]])

x = np.array([[2.499],
               [2.499],
               [-5.1]])

In [46]:
FeasibilityEllipsoidMethod(A,x)

A valid solution is: 
 [[2.51006966]
 [2.53069097]
 [0.        ]]
